# Авторазметка стилей: zero-shot → BERT

Ноутбук строит автоматический pipeline для разметки корпуса:
1. Небольшой seed датасет собирается с помощью zero-shot/LLM (через HuggingFace).
2. На seed обучается компактная BERT-модель, которая переносит метки на весь корпус.
3. Отбираются сомнительные примеры для активного уточнения (ручная проверка или вызов LLM второй волной).

## Импорты и подготовка окружения

Убедитесь, что установлены пакеты `transformers`, `accelerate`, `datasets`, `torch`, `tqdm`, `scikit-learn`. В VRAM ≈12 ГБ удобно работать с моделями до 110 M параметров (RuBERT, MiniLM и др.).

In [1]:
from __future__ import annotations

import json
import math
import os
import random
from dataclasses import dataclass
from pathlib import Path
from typing import Dict, List, Optional

import numpy as np
import pandas as pd
os.environ['TRANSFORMERS_NO_TF'] = '1'
import torch
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    Trainer,
    TrainingArguments,
    pipeline,
)

torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE

device(type='cuda')

## Конфигурация автозаметчика

Все артефакты складываем в `cache_boosted/autolabel`. Zero-shot модель можно заменить на любую подходящую для русского языка (`cointegrated/rubert-tiny2`, `MoravecLab/zero-shot-classifier-rus`, `joeddav/xlm-roberta-large-xnli`, и т. д.).

In [2]:
AUTOLABEL_DIR = Path("cache_boosted/autolabel")
AUTOLABEL_DIR.mkdir(parents=True, exist_ok=True)

# Настройки сгруппированы по смыслу. Меняйте значения из подписанных блоков,
# остальное оставлено по умолчанию для воспроизводимости.
config: Dict[str, object] = {
    # --- Данные ---
    "DATASET_CSV": Path("data/taiga_style_dataset.csv"),  # корпус из style_dataset_preparation.ipynb
    "TEXT_COLUMN": "text",
    "HINT_COLUMN": "style_hint_label",
    "HINT_NAME_COLUMN": "style_hint",
    "HINT_CONFIDENCE_COLUMN": "style_hint_confidence",

    # --- Подбор zero-shot ---
    "ZERO_SHOT_MODELS": [  # кандидаты; добавьте свои при необходимости
        "joeddav/xlm-roberta-large-xnli",
        "facebook/bart-large-mnli",
        "cointegrated/rubert-tiny2",
        "DeepPavlov/rubert-base-cased",
    ],
    "ZERO_SHOT_MODEL": "cointegrated/rubert-tiny2",  # будет переопределён по результатам оценки
    "ZERO_SHOT_SAMPLE_SIZE": 3000,  # сколько примеров берём для оценки кандидатов
    "ZERO_SHOT_THRESHOLD": 0.48,  # ↑ делает seed чище, но меньше (рекомендовано 0.45–0.6)

    # --- Обучение компактного BERT ---
    "LABEL_NAMES": {
        0: "разговорный стиль",
        1: "официально-деловой стиль",
    },
    "BERT_BASE_MODEL": "ai-forever/ruBert-base",  # можно заменить на любой rus BERT
    "MAX_LENGTH": 256,  # длина токенов для BERT (256 для экономии VRAM, 512 для макс. качества)
    "TRAIN_SIZE": 0.85,  # доля train от seed (остальное -> val)

    # --- Повторяемость и фильтры ---
    "SEED": 2024,
    "PSEUDO_MIN_CONF": 0.70,  # отбрасываем псевдометки ниже порога перед сохранением
}
random.seed(config["SEED"])
np.random.seed(config["SEED"])
torch.manual_seed(config["SEED"])
config



{'DATASET_CSV': PosixPath('data/taiga_style_dataset.csv'),
 'TEXT_COLUMN': 'text',
 'HINT_COLUMN': 'style_hint_label',
 'HINT_NAME_COLUMN': 'style_hint',
 'HINT_CONFIDENCE_COLUMN': 'style_hint_confidence',
 'ZERO_SHOT_MODELS': ['joeddav/xlm-roberta-large-xnli',
  'facebook/bart-large-mnli',
  'cointegrated/rubert-tiny2',
  'DeepPavlov/rubert-base-cased'],
 'ZERO_SHOT_MODEL': 'cointegrated/rubert-tiny2',
 'ZERO_SHOT_SAMPLE_SIZE': 3000,
 'ZERO_SHOT_THRESHOLD': 0.48,
 'LABEL_NAMES': {0: 'разговорный стиль', 1: 'официально-деловой стиль'},
 'BERT_BASE_MODEL': 'ai-forever/ruBert-base',
 'MAX_LENGTH': 256,
 'TRAIN_SIZE': 0.85,
 'SEED': 2024,
 'PSEUDO_MIN_CONF': 0.7}

In [3]:
if not config["DATASET_CSV"].exists():
    raise FileNotFoundError(
        f"Не найден {config['DATASET_CSV']}. Перед авторазметкой выполните подготовку корпуса."
    )

raw_df = pd.read_csv(config["DATASET_CSV"], encoding="utf-8")
raw_df = raw_df.dropna(subset=[config["TEXT_COLUMN"]]).copy()
raw_df[config["TEXT_COLUMN"]] = raw_df[config["TEXT_COLUMN"]].astype(str)

hint_col = config.get("HINT_COLUMN")
if hint_col:
    if hint_col in raw_df.columns:
        raw_df[hint_col] = raw_df[hint_col].fillna(-1).astype(int)
    else:
        raw_df[hint_col] = -1

hint_name_col = config.get("HINT_NAME_COLUMN")
if hint_name_col:
    if hint_name_col in raw_df.columns:
        raw_df[hint_name_col] = raw_df[hint_name_col].fillna("").astype(str)
    else:
        raw_df[hint_name_col] = ""

hint_conf_col = config.get("HINT_CONFIDENCE_COLUMN")
if hint_conf_col:
    if hint_conf_col in raw_df.columns:
        raw_df[hint_conf_col] = raw_df[hint_conf_col].fillna(0.0).astype(float)
    else:
        raw_df[hint_conf_col] = 0.0

print(f"Корпус: {len(raw_df)} сегментов")
if hint_col:
    hint_counts = raw_df[hint_col].value_counts(dropna=False).sort_index()
    display(hint_counts.rename("rows"))
raw_df.head()


Корпус: 16000 сегментов


style_hint_label
0    8000
1    8000
Name: rows, dtype: int64

,text,source_archive,source_file,meta_id,meta_title,meta_languages,meta_textid,meta_rubric,meta_region,meta_date,meta_tags,style_hint_label,style_hint,style_hint_confidence,style_hint_source
0,Кто против ЕР с сотней Казаков подъедем пошука...,social.tar.gz,home/tsha/social/texts/vktexts.txt#entry162588,162588,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,разговорный стиль,0.60,archive
1,"На 36-й минуте игры с ""Зенитом"" во вратаря ""Ди...",Interfax.tar.gz,home/tsha/Interfax/texts/sport277931.txt,sport277931,Вратарю &quot;Динамо&quot; Шунину разрешили тр...,NaN,sport277931,Спорт,NaN,2012-11-27,футбол,1,официально-деловой стиль,0.75,archive
2,Пора прекратить размахивать Ядерной Дубиной!,social.tar.gz,home/tsha/social/texts/vktexts.txt#entry284914,284914,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,разговорный стиль,0.60,archive
3,"Теги: футбол, Лига чемпионов, Спартак, травма",Interfax.tar.gz,home/tsha/Interfax/texts/sport261719.txt,sport261719,"Футболисты &quot;Спартака&quot; Ромулу, Пареха...",NaN,sport261719,Спорт,NaN,2012-08-22,футбол,1,официально-деловой стиль,0.75,archive
4,Предвыборный штаб кандидата в президенты Влади...,Interfax.tar.gz,home/tsha/Interfax/texts/russia232337.txt,russia232337,Предвыборный штаб Путина в оставшееся до 4 мар...,NaN,russia232337,В России,NaN,2012-02-23,Выборы,1,официально-деловой стиль,0.75,archive


## Подбор zero-shot модели

Сравним несколько моделей по точности на небольшом подмножестве корпуса и выберем лучшую для генерации seed-меток.


In [4]:
label_names = list(config["LABEL_NAMES"].values())
inverse_label = {name: idx for idx, name in config["LABEL_NAMES"].items()}

candidate_models = config.get("ZERO_SHOT_MODELS", [])
if not candidate_models:
    raise ValueError("Список ZERO_SHOT_MODELS в конфиге пуст.")

hint_col = config.get("HINT_COLUMN")
if hint_col is None or hint_col not in raw_df.columns:
    raise RuntimeError("Нет эвристических подсказок (style hints) для оценки zero-shot моделей.")

hint_mask = raw_df[hint_col] >= 0
if not hint_mask.any():
    raise RuntimeError("Эвристические подсказки отсутствуют, нечем калибровать zero-shot модель.")

eval_pool = raw_df[hint_mask].copy()
num_classes = max(1, eval_pool[hint_col].nunique())
per_class = max(1, config["ZERO_SHOT_SAMPLE_SIZE"] // num_classes)
eval_df = (
    eval_pool.groupby(hint_col, group_keys=False)
    .apply(lambda g: g.sample(min(len(g), per_class), random_state=config["SEED"]))
    .reset_index(drop=True)
)
eval_df = eval_df.sample(min(len(eval_df), config["ZERO_SHOT_SAMPLE_SIZE"]), random_state=config["SEED"]).reset_index(drop=True)

evaluation_results = []
device_index = 0 if torch.cuda.is_available() else -1

for model_name in candidate_models:
    print(f"Модель: {model_name}")
    try:
        clf = pipeline("zero-shot-classification", model=model_name, device=device_index)
    except Exception as exc:  # noqa: BLE001
        print(f"[WARN] Пропускаем модель {model_name}: {exc}")
        evaluation_results.append(
            {
                "model": model_name,
                "accuracy": np.nan,
                "macro_f1": np.nan,
                "mean_confidence": np.nan,
                "samples": 0,
                "error": str(exc),
            }
        )
        continue
    preds = []
    scores = []
    batch_size = 8
    total_batches = math.ceil(len(eval_df) / batch_size)
    for start in tqdm(range(0, len(eval_df), batch_size), total=total_batches, desc="Inference", unit="batch"):
        batch = eval_df.iloc[start : start + batch_size]
        outputs = clf(list(batch[config["TEXT_COLUMN"]]), candidate_labels=label_names)
        if isinstance(outputs, dict):
            outputs = [outputs]
        for out in outputs:
            label_name = out["labels"][0]
            label_idx = inverse_label.get(label_name)
            if label_idx is None:
                continue
            preds.append(label_idx)
            scores.append(float(out["scores"][0]))
    if not preds:
        del clf
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        continue
    acc = accuracy_score(eval_df[hint_col].iloc[: len(preds)], preds)
    macro_f1 = f1_score(eval_df[hint_col].iloc[: len(preds)], preds, average="macro")
    mean_conf = float(np.mean(scores)) if scores else 0.0
    evaluation_results.append(
        {
            "model": model_name,
            "accuracy": acc,
            "macro_f1": macro_f1,
            "mean_confidence": mean_conf,
            "samples": len(preds),
        }
    )
    del clf
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

evaluation_df = pd.DataFrame(evaluation_results)
if not evaluation_df.empty:
    evaluation_df = evaluation_df.sort_values("macro_f1", ascending=False)
display(evaluation_df)

valid_df = evaluation_df.dropna(subset=["macro_f1"]) if not evaluation_df.empty else evaluation_df
if not valid_df.empty:
    best_model_name = valid_df.iloc[0]["model"]
    print(f"Лучший zero-shot (по согласию с эвристикой): {best_model_name}")
    config["ZERO_SHOT_MODEL"] = best_model_name
else:
    print("Не удалось подобрать zero-shot модель. Проверьте список ZERO_SHOT_MODELS.")


/tmp/ipykernel_117648/2710994355.py:21: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: g.sample(min(len(g), per_class), random_state=config["SEED"]))


Модель: joeddav/xlm-roberta-large-xnli


config.json:   0%|          | 0.00/734 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Cancellation requested; stopping current tasks.


KeyboardInterrupt: 

## Seed через zero-shot / LLM

1. Сэмплируем сбалансированный seed.
2. Прогоняем через zero-shot модель (`transformers.pipeline`).
3. Фильтруем по порогу уверенности и сохраняем в `seed_labels.csv`.

In [ ]:
hint_col = config.get("HINT_COLUMN")
hint_mask = (raw_df[hint_col] >= 0) if hint_col else pd.Series(False, index=raw_df.index)
target_seed = min(len(raw_df), config["ZERO_SHOT_SAMPLE_SIZE"])

if hint_col and hint_mask.any():
    classes = max(1, raw_df.loc[hint_mask, hint_col].nunique())
    per_class = max(1, target_seed // classes)
    seed_df = (
        raw_df.loc[hint_mask]
        .groupby(hint_col, group_keys=False)
        .apply(lambda g: g.sample(min(len(g), per_class), random_state=config["SEED"]))
        .reset_index(drop=True)
    )
    remaining = target_seed - len(seed_df)
    if remaining > 0:
        remainder = raw_df.loc[~hint_mask]
        if not remainder.empty:
            extra = remainder.sample(min(len(remainder), remaining), random_state=config["SEED"])
            seed_df = pd.concat([seed_df, extra], ignore_index=True)
else:
    seed_df = raw_df.sample(target_seed, random_state=config["SEED"]).reset_index(drop=True)

seed_df = seed_df.sample(frac=1.0, random_state=config["SEED"]).reset_index(drop=True)
print(f"Seed размер: {len(seed_df)}")
seed_df.head()


Seed размер: 3000


/tmp/ipykernel_30888/1159421413.py:11: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: g.sample(min(len(g), per_class), random_state=config["SEED"]))


,text,source_archive,source_file,meta_id,meta_title,meta_languages,meta_textid,meta_rubric,meta_region,meta_date,meta_tags,style_hint_label,style_hint,style_hint_confidence,style_hint_source
0,Вы нихуя не понимаете в Ельцине! Он был эпичен...,social.tar.gz,home/tsha/social/texts/vktexts.txt#entry25570,25570,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,разговорный стиль,0.6,archive
1,"Петушки, видящие в трампе эдакого американског...",social.tar.gz,home/tsha/social/texts/vktexts.txt#entry241406,241406,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,разговорный стиль,0.6,archive
2,"Едро, лдпр, комуняги и справедливоросы 45%, 50...",social.tar.gz,home/tsha/social/texts/vktexts.txt#entry259224,259224,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,разговорный стиль,0.6,archive
3,Я еще предлагаю что бы активисты закидали пись...,social.tar.gz,home/tsha/social/texts/vktexts.txt#entry184724,184724,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,разговорный стиль,0.6,archive
4,"Ну в ""Единой России"" хотя бы шевеления происхо...",social.tar.gz,home/tsha/social/texts/vktexts.txt#entry68069,68069,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,разговорный стиль,0.6,archive


In [ ]:
zero_shot = None
try:
    zero_shot = pipeline(
        "zero-shot-classification",
        model=config["ZERO_SHOT_MODEL"],
        device=0 if torch.cuda.is_available() else -1,
    )
except Exception as exc:  # noqa: BLE001
    print(f"[WARN] Не удалось загрузить zero-shot модель {config['ZERO_SHOT_MODEL']}: {exc}")
zero_shot


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0
Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.


In [ ]:
predictions: List[Dict[str, object]] = []

if zero_shot is None:
    print("Zero-shot модель не загружена, пропускаем инференс.")
    pred_df = pd.DataFrame(predictions)
else:
    batch_size = 8
    total_batches = math.ceil(len(seed_df) / batch_size)
    hint_col = config.get("HINT_COLUMN")
    hint_name_col = config.get("HINT_NAME_COLUMN")
    hint_conf_col = config.get("HINT_CONFIDENCE_COLUMN")
    for start in tqdm(
        range(0, len(seed_df), batch_size),
        total=total_batches,
        desc="Zero-shot inference",
        unit="batch",
    ):
        batch = seed_df.iloc[start : start + batch_size]
        outputs = zero_shot(list(batch[config["TEXT_COLUMN"]]), candidate_labels=label_names)
        if isinstance(outputs, dict):  # когда размер batch=1
            outputs = [outputs]
        for row, out in zip(batch.itertuples(index=False), outputs):
            label_name = out["labels"][0]
            pred_label = inverse_label.get(label_name)
            if pred_label is None:
                continue
            pred_score = float(out["scores"][0])
            second = out["labels"][1] if len(out["labels"]) > 1 else None
            hint_label = getattr(row, hint_col, -1) if hint_col else -1
            hint_name = getattr(row, hint_name_col, "") if hint_name_col else ""
            if hint_conf_col and hasattr(row, hint_conf_col):
                hint_conf = float(getattr(row, hint_conf_col))
            else:
                hint_conf = None
            hint_agree = (hint_label == pred_label) if hint_label is not None and hint_label >= 0 else None
            predictions.append(
                {
                    "text": getattr(row, config["TEXT_COLUMN"]),
                    "pred_label_name": label_name,
                    "pred_label": pred_label,
                    "pred_score": pred_score,
                    "second_best": second,
                    "hint_label": hint_label,
                    "hint_name": hint_name,
                    "hint_confidence": hint_conf,
                    "hint_agrees": hint_agree,
                    "source_archive": getattr(row, "source_archive", ""),
                }
            )
    pred_df = pd.DataFrame(predictions)

pred_df.head()


Zero-shot inference:   0%|          | 0/375 [00:00<?, ?batch/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


,text,pred_label_name,pred_label,pred_score,second_best,hint_label,hint_name,hint_confidence,hint_agrees,source_archive
0,Вы нихуя не понимаете в Ельцине! Он был эпичен...,разговорный стиль,0,0.515588,официально-деловой стиль,0,разговорный стиль,0.6,True,social.tar.gz
1,"Петушки, видящие в трампе эдакого американског...",разговорный стиль,0,0.503155,официально-деловой стиль,0,разговорный стиль,0.6,True,social.tar.gz
2,"Едро, лдпр, комуняги и справедливоросы 45%, 50...",разговорный стиль,0,0.507794,официально-деловой стиль,0,разговорный стиль,0.6,True,social.tar.gz
3,Я еще предлагаю что бы активисты закидали пись...,разговорный стиль,0,0.506651,официально-деловой стиль,0,разговорный стиль,0.6,True,social.tar.gz
4,"Ну в ""Единой России"" хотя бы шевеления происхо...",разговорный стиль,0,0.502083,официально-деловой стиль,0,разговорный стиль,0.6,True,social.tar.gz


In [ ]:

threshold = config["ZERO_SHOT_THRESHOLD"]
seed_filtered = pred_df[pred_df["pred_score"] >= threshold].copy()
print(f"Фильтр по уверенности {threshold:.2f}: оставлено {len(seed_filtered)} примеров")
if not seed_filtered.empty:
    group_cols = ["pred_label_name", "hint_agrees"]
    agg = (
        seed_filtered.groupby(group_cols)["pred_score"]
        .agg(["count", "mean", "min", "max"])
        .reset_index()
    )
    display(agg)

seed_path = AUTOLABEL_DIR / "seed_labels.csv"
seed_filtered.to_csv(seed_path, index=False)
print(f"Seed сохранён в {seed_path}")


Фильтр по уверенности 0.48: оставлено 3000 примеров


,pred_label_name,hint_agrees,count,mean,min,max
0,официально-деловой стиль,False,222,0.503119,0.500049,0.514017
1,официально-деловой стиль,True,245,0.504918,0.500059,0.520886
2,разговорный стиль,False,1255,0.505662,0.500013,0.528475
3,разговорный стиль,True,1278,0.505267,0.500003,0.526951


Seed сохранён в cache_boosted/autolabel/seed_labels.csv


## Fine-tuning RuBERT на seed

Теперь обучаем компактный классификатор, чтобы перенести стилистические метки на весь корпус.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

seed_for_train = seed_filtered.copy()
autolabel_ready = True

if "pred_label" in seed_for_train.columns and not seed_for_train.empty:
    seed_for_train["label"] = seed_for_train["pred_label"].astype(int)
else:
    seed_for_train["label"] = pd.Series(index=seed_for_train.index, dtype=int)

label_counts = seed_for_train["label"].value_counts()
class_count = int(len(label_counts))
if seed_for_train.empty:
    print("[WARN] Seed пуст: zero-shot не дал ни одного примера. Обучение пропускаем.")
    autolabel_ready = False
elif class_count < 2:
    print("[WARN] Seed содержит менее двух классов. Невозможно обучить классификатор, пропускаем обучение.")
    autolabel_ready = False
elif label_counts.min() < 2:
    print("[WARN] Для хотя бы одного класса слишком мало примеров для стратификации. Обучение пропускаем.")
    autolabel_ready = False
else:
    try:
        train_df, val_df = train_test_split(
            seed_for_train,
            test_size=1 - config["TRAIN_SIZE"],
            stratify=seed_for_train["label"],
            random_state=config["SEED"],
        )
        train_df = train_df.reset_index(drop=True)
        val_df = val_df.reset_index(drop=True)
        print(f"Train: {len(train_df)}, Val: {len(val_df)}")
    except ValueError as exc:  # например, слишком мало объектов на класс
        print(f"[WARN] Не удалось разбить seed: {exc}")
        autolabel_ready = False

if not autolabel_ready:
    train_df = seed_for_train.reset_index(drop=True)
    val_df = seed_for_train.iloc[0:0].reset_index(drop=True)



Train: 2549, Val: 451


In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer
from typing import Dict

tokenizer = AutoTokenizer.from_pretrained(config["BERT_BASE_MODEL"])

@dataclass
class StyleDataset:
    df: pd.DataFrame
    tokenizer: AutoTokenizer
    text_column: str
    label_column: str
    max_length: int

    def __len__(self) -> int:
        return len(self.df)

    def __getitem__(self, idx: int) -> Dict[str, torch.Tensor]:
        row = self.df.iloc[idx]
        encoded = self.tokenizer(
            row[self.text_column],
            truncation=True,
            max_length=self.max_length,
        )
        encoded["labels"] = int(row[self.label_column])
        return {key: torch.tensor(val) for key, val in encoded.items()}

train_dataset = StyleDataset(train_df.reset_index(drop=True), tokenizer, config["TEXT_COLUMN"], "label", config["MAX_LENGTH"])
val_dataset = StyleDataset(val_df.reset_index(drop=True), tokenizer, config["TEXT_COLUMN"], "label", config["MAX_LENGTH"])

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)



In [ ]:
if not autolabel_ready or train_df.empty:
    print("[WARN] Недостаточно seed-данных для обучения BERT. Пропускаем тренировку.")
    trainer = None
else:
    model = AutoModelForSequenceClassification.from_pretrained(
        config["BERT_BASE_MODEL"],
        num_labels=len(config["LABEL_NAMES"]),
    )

    from inspect import signature

    # TrainingArguments API меняется между версиями transformers; подбираем поддерживаемые ключи
    sig_params = set(signature(TrainingArguments.__init__).parameters)

    def _set_first(names, value, target):
        for name in names:
            if name in sig_params:
                target[name] = value
                break

    training_kwargs = {
        "output_dir": str(AUTOLABEL_DIR / "rubert_autolabel"),
    }

    _set_first(("evaluation_strategy", "eval_strategy"), "epoch", training_kwargs)
    _set_first(("save_strategy", "save_strategy"), "epoch", training_kwargs)

    for key, value in [
        ("learning_rate", 2e-5),
        ("per_device_train_batch_size", 16),
        ("per_device_eval_batch_size", 16),
        ("num_train_epochs", 3),
        ("weight_decay", 0.01),
        ("report_to", "none"),
        ("load_best_model_at_end", True),
        ("metric_for_best_model", "macro_f1"),
    ]:
        if key in sig_params:
            training_kwargs[key] = value

    training_args = TrainingArguments(**training_kwargs)

    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        preds = logits.argmax(axis=-1)
        return {
            "accuracy": accuracy_score(labels, preds),
            "macro_f1": f1_score(labels, preds, average="macro"),
        }

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

trainer


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ai-forever/ruBert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_30888/771381291.py:51: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
if trainer is None:
    print("[WARN] Тренировочный пайплайн не инициализирован: нет данных для обучения.")
else:
    print("Trainer готов. Раскомментируйте строку ниже, чтобы запустить обучение.")
    trainer.train()
    # После обучения можно сохранить модель:
    trainer.save_model(AUTOLABEL_DIR / "rubert_autolabel/best_model")
    tokenizer.save_pretrained(AUTOLABEL_DIR / "rubert_autolabel/best_model")



Trainer готов. Раскомментируйте строку ниже, чтобы запустить обучение.


Epoch,Training Loss,Validation Loss,Accuracy,Macro F1
1,No log,0.397663,0.822616,0.572390
2,No log,0.423924,0.824834,0.587804
3,No log,0.473974,0.818182,0.605904


## Применение модели и активное обучение

- Применяем обученную модель ко всему корпусу.
- Сохраняем псевдометки + вероятность.
- Формируем пул сомнительных текстов для дополнительной проверки.

In [ ]:
from typing import TYPE_CHECKING, Dict, List
import math
import numpy as np
import torch
from tqdm.auto import tqdm
from transformers import AutoModelForSequenceClassification

if TYPE_CHECKING:
    from transformers import AutoTokenizer

# Глобальные объекты задаются в предыдущих ячейках; типы нужны для анализаторов
config: Dict[str, object]
tokenizer: "AutoTokenizer"


def predict_proba(model: AutoModelForSequenceClassification, texts: List[str], batch_size: int = 64) -> np.ndarray:
    model.eval()
    probs: List[np.ndarray] = []
    with torch.no_grad():
        total_batches = math.ceil(len(texts) / batch_size)
        for start in tqdm(
            range(0, len(texts), batch_size),
            total=total_batches,
            desc="Model inference",
            unit="batch",
        ):
            batch = texts[start : start + batch_size]
            enc = tokenizer(
                batch,
                truncation=True,
                max_length=config["MAX_LENGTH"],
                padding=True,
                return_tensors="pt",
            ).to(model.device)
            logits = model(**enc).logits
            probs.append(logits.softmax(dim=-1).cpu().numpy())
    return np.concatenate(probs, axis=0)

# Пример использования после обучения: считаем уверенности, фильтруем и сохраняем.
full_probs = predict_proba(trainer.model, raw_df[config["TEXT_COLUMN"]].tolist())

pseudo_df = raw_df.copy()
pseudo_df["pseudo_label"] = full_probs.argmax(axis=1)
pseudo_df["pseudo_confidence"] = full_probs.max(axis=1)

manual_threshold = 0.6  # записи ниже этого порога удобнее отдать на ручную проверку
low_conf_mask = pseudo_df["pseudo_confidence"] < manual_threshold
if low_conf_mask.any():
    review_path = AUTOLABEL_DIR / "manual_review_candidates.csv"
    pseudo_df.loc[low_conf_mask].to_csv(review_path, index=False)
    print(f"[INFO] {low_conf_mask.sum()} примеров сохранены для ручной проверки → {review_path}")

min_conf = float(config.get("PSEUDO_MIN_CONF", 0.0) or 0.0)
if min_conf > 0:
    before = len(pseudo_df)
    pseudo_df = pseudo_df[pseudo_df["pseudo_confidence"] >= min_conf].copy()
    print(f"[INFO] Отфильтрованы слабые псевдометки: {before} → {len(pseudo_df)} (порог {min_conf:.2f}).")

pseudo_path = AUTOLABEL_DIR / "pseudo_labels.csv"
pseudo_df.to_csv(pseudo_path, index=False)
print(f"Псевдометки сохранены в {pseudo_path}")



Model inference:   0%|          | 0/250 [00:00<?, ?batch/s]

[INFO] 579 примеров сохранены для ручной проверки → cache_boosted/autolabel/manual_review_candidates.csv
[INFO] Отфильтрованы слабые псевдометки: 16000 → 14758 (порог 0.70).
Псевдометки сохранены в cache_boosted/autolabel/pseudo_labels.csv


In [ ]:
def select_uncertain_examples(df: pd.DataFrame, confidence_col: str, top_n: int = 200) -> pd.DataFrame:
    """Выбираем тексты с минимальной уверенностью для активного уточнения."""
    if confidence_col not in df.columns:
        raise KeyError(f"Колонка {confidence_col} отсутствует")
    return df.nsmallest(top_n, confidence_col)

# Пример использования после псевдометок:
uncertain = select_uncertain_examples(pseudo_df, 'pseudo_confidence', top_n=300)
uncertain_path = AUTOLABEL_DIR / 'uncertain_samples.csv'
uncertain.to_csv(uncertain_path, index=False)
print(f'Сомнительные примеры сохранены в {uncertain_path}. Используйте LLM/ручную проверку.')

Сомнительные примеры сохранены в cache_boosted/autolabel/uncertain_samples.csv. Используйте LLM/ручную проверку.


## Что дальше

1. Опционально дособрать seed через более мощную LLM и объединить с текущим.
2. Запустить ячейки fine-tuning и псевдометок, получить финальный корпус для основной классификации.
3. Повторять активное обучение до стабилизации метрик.
4. Все артефакты лежат в `cache_boosted/autolabel`.